# Quantum Sticks

### The Classical Version
The classical version of the "Sticks" game can be described as follows:
* There are a total of n sticks. 
* Each player can either pick 1, 2 or 3 sticks per turn. 
* The player with the current turn and no more sticks to pick loses. 

### Let's Quantumize It

We propose a quantum version of the same game- the description is as follows:
* There are a total of n sticks
* Each player can choose to either pick 1,2 or 3 sticks per turn. However, the architecture of the game may/may not allow those many sticks to be picked. The total number of sticks picked may increase/decrease. Goofy.

In order to achieve this 'goofy action', we have the below:
* We create n qubits- half of these are initialized to |0>, the others to |1>
* The user can choose to put some qubits in superposition
* We entangle some pairs of qubits in the back-end
* When a player picks m sticks, we measure those many sticks (and if there are any other sticks which are entangled to the selected sticks). Depending on the measurement outcome, we calculate how many sticks were actually picked.


#### How the game functions

In [1]:
from qiskit import *
import numpy as np
import pyfiglet
import time
import cowsay
import termcolor
import random
from IPython.display import clear_output

In [2]:
#Game title
termcolor.cprint(pyfiglet.figlet_format("Quantum Sticks" ),'green', attrs=['bold'])
#Authors
termcolor.cprint(pyfiglet.figlet_format(" Jaime  McCarthy ", font = "digital" ),'blue', 'on_grey')
termcolor.cprint(pyfiglet.figlet_format(" Kaushambi  Gujral ", font = "digital"),'magenta', 'on_grey')
termcolor.cprint(pyfiglet.figlet_format(" Lee  Hoang ", font = "digital"),'yellow', 'on_grey')

  ___                    _                     ____  _   _      _        
 / _ \ _   _  __ _ _ __ | |_ _   _ _ __ ___   / ___|| |_(_) ___| | _____ 
| | | | | | |/ _` | '_ \| __| | | | '_ ` _ \  \___ \| __| |/ __| |/ / __|
| |_| | |_| | (_| | | | | |_| |_| | | | | | |  ___) | |_| | (__|   <\__ \
 \__\_\\__,_|\__,_|_| |_|\__|\__,_|_| |_| |_| |____/ \__|_|\___|_|\_\___/
                                                                         

 +-+-+-+-+-+  +-+-+-+-+-+-+-+-+ 
 |J|a|i|m|e|  |M|c|C|a|r|t|h|y| 
 +-+-+-+-+-+  +-+-+-+-+-+-+-+-+ 

 +-+-+-+-+-+-+-+-+-+  +-+-+-+-+-+-+ 
 |K|a|u|s|h|a|m|b|i|  |G|u|j|r|a|l| 
 +-+-+-+-+-+-+-+-+-+  +-+-+-+-+-+-+ 

 +-+-+-+  +-+-+-+-+-+ 
 |L|e|e|  |H|o|a|n|g| 
 +-+-+-+  +-+-+-+-+-+ 



**Computer Strategies**

In [69]:
class Strategy:
    def __init__(self, strategy):
        self.strategy = strategy
        self.constant = random.randint(1,2)
        self.balance = 1
    
    def constant_strategy(self):
        return self.constant #always pick the same number of sticks
    
    def balanced_strategy(self):
        x = self.balance
        self.balance +=1
        return x % 3
    
    def classical_strategy(self, numSticks):
        if numSticks%4 == 3:
            return 2
        elif numSticks%4 == 2:
            return 1
        elif numSticks%4 == 1:
            return 1
        elif numSticks%4 == 0:
            return 3
        
    def q_random_strategy(self, sticks):
        max_sticks = min(3, sticks)
        bits = max_sticks.bit_length()
        print("bits = ", bits)
        qc = QuantumCircuit(bits) 
        qc.reset(0)
        qc.barrier()

        for i in range(bits):
            qc.h(i) # Hadamard (H)
            qc.x(i) # X gate
            qc.y(i) # Y gate

        qc.barrier()
        qc.measure_all()
        backend = Aer.get_backend('aer_simulator')
        job = backend.run(qc, shots=1, memory=True)
        output = job.result().get_memory()[0]
        print("output = ", output)
        return min(max_sticks, int(output, 2))
        
    def pick_sticks(self, sticks):
        if self.strategy == 1:
            return self.constant_strategy()
        elif self.strategy == 2:
            return self.balanced_strategy()
        elif self.strategy == 3:
            return self.classical_strategy(sticks)
        else:
            return self.q_random_strategy(sticks)

In [ ]:
'''

print("Testing computer's strategies")


print("Strategy 1")
s = Strategy(1)
sticks_remaining = 12
x = s.pick_sticks(sticks_remaining)
print(x)
sticks_remaining -= x
print(s.pick_sticks(sticks_remaining))


print("Strategy 2")
s = Strategy(2)
sticks_remaining = 12
x = s.pick_sticks(sticks_remaining)
print(x)
sticks_remaining -= x
print(s.pick_sticks(sticks_remaining))


print("Strategy 3")
s = Strategy(3)
sticks_remaining = 12
x = s.pick_sticks(sticks_remaining)
print(x)
sticks_remaining -= x
print(s.pick_sticks(sticks_remaining))



print("Strategy 4")
s = Strategy(4)
sticks_remaining = 12
x = s.pick_sticks(sticks_remaining)
print(x)
sticks_remaining -= x
print(s.pick_sticks(sticks_remaining))

'''

In [70]:
# Can you win the game with some probability p?

class QuantumSticks:
    
    def __init__(self):
        self.welcome()
        #data members
        self.turn = True
        self.min_sticks = 5 
        self.max_sticks = 25 
        self.level = 0
        self.n = self.level + self.min_sticks
        self.qr = None
        self.cr = None
        self.qc = None
        self.picked_sticks = []
        self.unpicked_sticks = (np.arange(0, self.n-1, 1, dtype=int)).tolist()
        self.entanglement = False
        self.step = 0
        self.entagled_states = {}
        self.strategy = None
        
    #---------------------
    #Utility Methods start
    #---------------------
    def clear(self, sleep_time):
        time.sleep(sleep_time)
        clear_output(wait=True)
        
    def slow_display(self, msg):
        for x in range (0,4):  
            b = msg + "." * x
            print (b, end="\r")
            time.sleep(0.8)
        print ("\n")
        
    #---------------------

    
    #---------------------
    # Creating Circuits 
    # For
    # Different Levels
    #---------------------

    def game_circuit(self):
        m = self.n
        l = self.level
        step = 0
        
        if l < 5:
            step = 1
            self.strategy = Strategy(1)
        elif l < 10:
            step = 4
            self.strategy = Strategy(2)
            
        elif l < 15:
            step = 3
            self.entanglement = True
            self.strategy = Strategy(3)
            
        else:
            step = 2
            self.entanglement = True
            self.strategy = Strategy(4)
            
        self.qr = QuantumRegister(m)
        self.cr = ClassicalRegister(1)
        self.qc = QuantumCircuit(self.qr, self.cr)
        self.qc.reset(range(m))
        for i in range(0, m, step):
            self.qc.x(i)
        self.n = m
        self.step = step
        
        
    def measure_specific(self, l):
        measurements = []
        for register in l:
            self.qc.measure(self.qr[register], self.cr)
            backend = Aer.get_backend('aer_simulator')
            job = backend.run(self.qc, shots=1024, memory=True)
            result = job.result()
            counts = result.get_counts(self.qc)
            zeros = counts.get('0', 0)
            ones = counts.get('1', 0)
            if int(zeros) > int(ones):
                measurements.append(0)
            else:
                measurements.append(1)
        return measurements  
    
    
    '''
        Depending upon the level,
        choose the complexity of
        the circuit
    '''
    def entangle(self):
        return 0
    '''
        Apply cnot to alternate qubits
    '''    
    def cnot_alternate(self):
        for i in range(0, self.n, 2):
            if i > self.n:
                self.qc.cx(i,i+1)    
        
    def draw_circuit(self):
        print(self.qc)     
        
    ''' Puts m sticks into superposition
    '''
    def initialize_sticks(self, superpositions): 
        print(superpositions)
        self.qc.barrier()
        
        for s in superpositions:
            sup = int(s) - 1
            if sup >= self.n or sup<0:
                continue
            self.qc.h(sup)
        
        self.qc.barrier()
          
        
    def pick_sticks(self, m):
        total_m = 0
        if len(self.unpicked_sticks) == 0:
            return -999
        for i in range(0, m):
            el = self.unpicked_sticks[0]
            if self.entanglement == False:
                # no entanglement
                # measure the qubit
                # return answer
                measurement = self.measure_specific([el])
                if measurement[0] == 1:
                    total_m +=1
                return total_m
            else:
                # you have entanglement
                if el%2==0:
                    el1 = el
                    el2 = el+1
                    measurements = self.measure_specific([el1, el2])
                    if measurements[0] == 1:
                        self.picked_sticks.append(el1)
                        self.unpicked_sticks.remove(el1)
                        total_m +=1
                    if measurements[1] == 1:
                        self.picked_sticks.append(el2)
                        self.unpicked_sticks.remove(el2)
                        total_m +=1
                else:
                    measurement = self.measure_specific([el])
                    if measurement[0] == 1:
                        total_m +=1
        return total_m #the number of sticks that were actually picked

    def level_input(self):
        print("Please select a level from 0-20")
        l = int(input()) % 21
        print("Level:",l)
        self.level = l
        self.n = self.level + self.min_sticks
        
    def play(self):
        self.level_input()
        self.clear(0.5)
        self.slow_display("Loading")
        self.clear(0.5)
        print("You have %2d sticks" % self.n)
        self.game_circuit()
        print("You can put some sticks into superposition.")
        print("Please enter comma separated values: such as 2,3,4,..")
        s = input().split(',')[:self.n]
        sups = [*set(s)]
        self.initialize_sticks(sups)
        self.draw_circuit()
        self.game()
        
        
    def game(self):
        strategy = self.strategy
        while self.n > 1:
            print('-------------------------------------------\nSticks left: %2d' %self.n)
            pick = 0
            if self.turn:
                # pick sticks
                print("Enter the number of sticks you want to pick- 1,2, or 3")
                p = int(input())
                pick = self.pick_sticks(p)
                print("Player picks %2d sticks" %p)
                self.turn = False
            else:
                # computer picks sticks
                c = strategy.pick_sticks(self.n)
                pick = self.pick_sticks(c)
                print("Computer picks %2d sticks" %c)
                self.turn = True
            
            if pick > self.n:
                pick = self.n-1
            self.n-= pick
            print("Sticks picked: %2d\n" %pick)
                
        if self.turn:         #n=1 & it's your turn
            cowsay.pig('You lose!')
        else:                 #n=1 & it's computer's turn
            cowsay.cow('You win!')
    
    def welcome(self):
        cowsay.cow('Welcome to Quantum Sticks')
        

In [71]:
q = QuantumSticks()
q.play()

You have  7 sticks
You can put some sticks into superposition.
Please enter comma separated values: such as 2,3,4,..
6,2,3
['2', '3', '6']
            ┌───┐ ░       ░ 
q20_0: ─|0>─┤ X ├─░───────░─
            ├───┤ ░ ┌───┐ ░ 
q20_1: ─|0>─┤ X ├─░─┤ H ├─░─
            ├───┤ ░ ├───┤ ░ 
q20_2: ─|0>─┤ X ├─░─┤ H ├─░─
            ├───┤ ░ └───┘ ░ 
q20_3: ─|0>─┤ X ├─░───────░─
            ├───┤ ░       ░ 
q20_4: ─|0>─┤ X ├─░───────░─
            ├───┤ ░ ┌───┐ ░ 
q20_5: ─|0>─┤ X ├─░─┤ H ├─░─
            ├───┤ ░ └───┘ ░ 
q20_6: ─|0>─┤ X ├─░───────░─
            └───┘ ░       ░ 
  c20: ═════════════════════
                            
-------------------------------------------
Sticks left:  7
Enter the number of sticks you want to pick- 1,2, or 3
2
Player picks  2 sticks
Sticks picked:  1

-------------------------------------------
Sticks left:  6
Computer picks  1 sticks
Sticks picked:  1

-------------------------------------------
Sticks left:  5
Enter the number of sticks you want to pick- 

In [72]:
q = QuantumSticks()
q.play()

You have 16 sticks
You can put some sticks into superposition.
Please enter comma separated values: such as 2,3,4,..
11,16,2,8,9,10,5
['8', '10', '9', '11', '5', '2', '16']
             ┌───┐ ░       ░ 
 q21_0: ─|0>─┤ X ├─░───────░─
             └───┘ ░ ┌───┐ ░ 
 q21_1: ─|0>───────░─┤ H ├─░─
                   ░ └───┘ ░ 
 q21_2: ─|0>───────░───────░─
             ┌───┐ ░       ░ 
 q21_3: ─|0>─┤ X ├─░───────░─
             └───┘ ░ ┌───┐ ░ 
 q21_4: ─|0>───────░─┤ H ├─░─
                   ░ └───┘ ░ 
 q21_5: ─|0>───────░───────░─
             ┌───┐ ░       ░ 
 q21_6: ─|0>─┤ X ├─░───────░─
             └───┘ ░ ┌───┐ ░ 
 q21_7: ─|0>───────░─┤ H ├─░─
                   ░ ├───┤ ░ 
 q21_8: ─|0>───────░─┤ H ├─░─
             ┌───┐ ░ ├───┤ ░ 
 q21_9: ─|0>─┤ X ├─░─┤ H ├─░─
             └───┘ ░ ├───┤ ░ 
q21_10: ─|0>───────░─┤ H ├─░─
                   ░ └───┘ ░ 
q21_11: ─|0>───────░───────░─
             ┌───┐ ░       ░ 
q21_12: ─|0>─┤ X ├─░───────░─
             └───┘ ░       ░ 
q21_13: ─|0>─────